In [ ]:
import numpy as np
import pandas as pd

# Transition matrices
gold_matrix = np.array([[1/4, 1/4, 1/2], [1/4, 1/2, 1/4], [1/2, 1/4, 1/4]])
e = np.linalg.eig(gold_matrix)

bc_matrix = np.array([[0.5, 0.45, 0.25], [0.25, 0.1, 0.25], [0.25, 0.45, 0.5]])
e2 = np.linalg.eig(bc_matrix)

[0.33333333 0.33333333 0.33333333]
EigResult(eigenvalues=array([ 1.  ,  0.25, -0.15]), eigenvectors=array([[-6.58145182e-01, -7.07106781e-01,  4.08248290e-01],
       [-3.65636212e-01,  1.24949658e-17, -8.16496581e-01],
       [-6.58145182e-01,  7.07106781e-01,  4.08248290e-01]]))
[0.39130435 0.2173913  0.39130435]


In [ ]:
# Strategy: buy low, sell high

# Allocation vector is of the form [cash, gold, bitcoin] in dollars
allocation = np.array([200, 400, 400])

n = 5 * 365 # number of days
cur = 1 # current day
prev_gold = allocation[1]
prev_bc = allocation[2]
move_threshold = 10

while cur <= n:
    # get current value from excel sheets
    # compare to previous values, if change is greater than move threshold, check states

    # States:
    # Make move matrix